In [ ]:
"""
To see if there is a faster way of generating surface skeletons
using what we learned from the meshparty module

"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")
#sys.path.append("../../meshAfterParty/meshAfterParty")

In [2]:
from importlib import reload
import os
from pathlib import Path
os.getcwd()

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)
import neuron_visualizations as nviz
import time

In [3]:
import system_utils as su

In [4]:
nru = reload(nru)
saved_file_location = "/notebooks/test_neurons/preprocessed_neurons/meshafterparty/12345_double_soma_meshafterparty"

with su.suppress_stdout_stderr():
    su.ignore_warnings()
    uncompressed_neuron = nru.decompress_neuron(filepath=saved_file_location,
                         original_mesh=saved_file_location)

In [5]:
uncompressed_neuron.decomposition_type

'meshafterparty'

# Get an example mesh that want to find surface skeleton of 

In [6]:
branch_idx = 0
ex_branch = uncompressed_neuron.concept_network.nodes["L0"]["data"].concept_network.nodes[branch_idx]["data"]

In [10]:
import skeleton_utils as sk

In [12]:
original_skeleton_time = time.time()
after_surface_skeleton = sk.generate_surface_skeleton(ex_branch.mesh.vertices,
                                                     ex_branch.mesh.faces)
print(f"Total time for surface skeletonization = {time.time() - original_skeleton_time}")
sk.graph_skeleton_and_mesh(other_meshes=[ex_branch.mesh],
                          other_skeletons=[after_surface_skeleton])

Total time for surface skeletonization = 16.837749481201172


# Meshparty way of generating a surface skeleton (mainly using mesh teaser)

In [17]:
from meshparty_skeletonize import *
def setup_root(mesh, is_soma_pt=None, soma_d=None, is_valid=None):
    """ function to find the root index to use for this mesh """
    if is_valid is not None:
        valid = np.copy(is_valid)
    else:
        valid = np.ones(len(mesh.vertices), np.bool)
    assert(len(valid) == mesh.vertices.shape[0])

    root = None
    # soma mode
    if is_soma_pt is not None:
        # pick the first soma as root
        assert(len(soma_d) == mesh.vertices.shape[0])
        assert(len(is_soma_pt) == mesh.vertices.shape[0])
        is_valid_root = is_soma_pt & valid
        valid_root_inds = np.where(is_valid_root)[0]
        if len(valid_root_inds) > 0:
            min_valid_root = np.nanargmin(soma_d[valid_root_inds])
            root = valid_root_inds[min_valid_root]
            root_ds, pred = sparse.csgraph.dijkstra(mesh.csgraph,
                                                    directed=False,
                                                    indices=root,
                                                    return_predecessors=True)
        else:
            start_ind = np.where(valid)[0][0]
            root, target, pred, dm, root_ds = utils.find_far_points(mesh,
                                                                    start_ind=start_ind)
        valid[is_soma_pt] = False

    if root is None:
        # there is no soma close, so use far point heuristic
        start_ind = np.where(valid)[0][0]
        root, target, pred, dm, root_ds = utils.find_far_points(
            mesh, start_ind=start_ind)
    valid[root] = False
    assert(np.all(~np.isinf(root_ds[valid])))
    return root, target,root_ds, pred, valid


In [30]:
import meshparty_skeletonize as m_sk
import meshparty

current_mesh = ex_branch.mesh

meshparty_skeleton_time = time.time()
branch_obj_tr_io  = meshparty.trimesh_io.Mesh(vertices = current_mesh.vertices,
                                   faces = current_mesh.faces,
                                   normals=current_mesh.face_normals)
root, target,root_ds, root_pred, valid = setup_root(branch_obj_tr_io)

current_path = utils.get_path(root,target,root_pred)

surface_sk_edges = np.vstack([current_path[:-1],current_path[1:]]).T
meshparty_surface_skeleton = branch_obj_tr_io.vertices[surface_sk_edges]
meshparty_surface_skeleton
print(f"Total time for surface skeletonization = {time.time() - meshparty_skeleton_time}")


Total time for surface skeletonization = 0.010904550552368164


In [31]:
sk.graph_skeleton_and_mesh(other_meshes=[ex_branch.mesh],
                          other_skeletons=[meshparty_surface_skeleton])

# Works wayyyyy faster

In [36]:
from skeleton_utils import downsample_skeleton

def generate_surface_skeleton(vertices,
                              faces, 
                              surface_samples=1000,
                              n_surface_downsampling=0,
                          print_flag=False):
    
    meshparty_skeleton_time = time.time()
    branch_obj_tr_io  = meshparty.trimesh_io.Mesh(vertices = vertices,
                                   faces=faces)
    
    root, target,root_ds, root_pred, valid = setup_root(branch_obj_tr_io)

    current_path = utils.get_path(root,target,root_pred)

    surface_sk_edges = np.vstack([current_path[:-1],current_path[1:]]).T
    meshparty_surface_skeleton = branch_obj_tr_io.vertices[surface_sk_edges]
    
    if print_flag: 
        print(f"Total time for surface skeletonization = {time.time() - meshparty_skeleton_time}")
    
    for i in range(n_surface_downsampling):
        meshparty_surface_skeleton = downsample_skeleton(meshparty_surface_skeleton)
    
    return meshparty_surface_skeleton

# Comparing 2 methods

In [39]:
meshparty_skeleton_time = time.time()
sk.generate_surface_skeleton(ex_branch.mesh.vertices,
                            ex_branch.mesh.faces)
print(f"Total time for surface skeletonization = {time.time() - meshparty_skeleton_time}")


Total time for surface skeletonization = 16.90719771385193


In [43]:
meshparty_skeleton_time = time.time()

new_skeleton = generate_surface_skeleton(ex_branch.mesh.vertices,
                              ex_branch.mesh.faces, 
                              surface_samples=1000,
                              n_surface_downsampling=4,
                          print_flag=False)
print(f"Total time for surface skeletonization = {time.time() - meshparty_skeleton_time}")


Total time for surface skeletonization = 0.013512134552001953


In [44]:
sk.graph_skeleton_and_mesh(other_meshes=[ex_branch.mesh],
                          other_skeletons=[new_skeleton])